In [1]:
using NLOptControl
using FastGaussQuadrature

In [2]:
Ni=2;Nck=[10,5]; t0=0.0; tf = 4.0;
# initialize node data TODO  make different PS methods available
taus_and_weights = [gaussradau(Nck[int]) for int in 1:Ni];
τ = [taus_and_weights[int][1] for int in 1:Ni];
ω = [taus_and_weights[int][2] for int in 1:Ni];
di, tm, ts, ωₛ = create_intervals(t0,tf,Ni,Nck,τ,ω);


In [3]:
function lepoly(n::Int64,x,derivative_option::Bool)
# lepoly  Legendre polynomial of degree n
    # y=lepoly(n,x) is the Legendre polynomial
    # The degree should be a nonnegative integer
    # The argument x should be on the closed interval [-1,1];
    # [dy,y]=lepoly(n,x) also returns the values of 1st-order
    #  derivative of the Legendre polynomial stored in dy
# Last modified on August 30, 2011
# Verified with the chart in http://keisan.casio.com/has10/SpecExec.cgi
  if !derivative_option
       if n==0 y=ones(size(x));  return y end
       if n==1 y=x; return y end
       polylst=ones(size(x)); poly=x;   # L_0(x)=1, L_1(x)=x
       polyn=0;
       for k=2:n                      # Three-term recurrence relation:
  	     polyn=((2*k-1)*x.*poly-(k-1)*polylst)/k; # kL_k(x)=(2k-1)xL_{k-1}(x)-(k-1)L_{k-2}(x)
         polylst=poly; poly=polyn;
       end
       y=polyn;
  else
     if n==0 y=ones(size(x)); dy=zeros(size(x)); return dy,y end
     if n==1 y=x; dy=ones(size(x)); return dy, y end

     # TODO MAKE SURE THIS IS CALCULATING SIZE correctly!! i.e. N=4 and size(N+1,N+1) = 1
      polylst=ones(size(x)); pderlst=zeros(size(x));poly=x; pder=ones(size(x));
       # L_0=1, L_0'=0, L_1=x, L_1'=1
      polyn=0;
      pdern=0;
      for k=2:n                          # Three-term recurrence relation:
        polyn=((2*k-1)*x.*poly-(k-1)*polylst)/k; # kL_k(x)=(2k-1)xL_{k-1}(x)-(k-1)L_{k-2}(x)
        pdern=pderlst+(2*k-1)*poly; # L_k'(x)=L_{k-2}'(x)+(2k-1)L_{k-1}(x)
     	  polylst=poly; poly=polyn;
    	  pderlst=pder; pder=pdern;
      end
      y=polyn; dy=pdern;
      return dy, y
  end

end


# the following code use

lepoly (generic function with 1 method)

In [4]:
#--------------------------------------------------------------------------
# Author: Huckleberry Febbo, Graduate Student, University of Michigan
# Date: 12/23/2015
# Last modifed on December 23, 2016 by Huckleberry Febbo
#--------------------------------------------------------------------------
# From a modified version of what is described in the paper (*) titled:
# "Legendre–Gauss–Lobatto Pseudo–spectral Method for One–Dimensional Advection–Diffusion Equation"
function MYD(τ::Vector{Float64},N::Int64)
  D = zeros(N+1,N+1)
  for i in 1:N+1
    for j in 1:N+1
      if i!=j
        D[i,j] = lepoly(N,τ[i],false)[1]/(lepoly(N,τ[j],false)[1]*(τ[i]-τ[j]));
      elseif (i==j && j==1)
        D[i,j] = -N*(N+1)/4;
      elseif (i==j && j==(N+1)) #TODO added the N+1 --> check this (what I modified from the paper(*))
        D[i,j] = N*(N+1)/4;
      else
        D[i,j] = 0;
      end
    end
  end
  return D
end


THIS IS THE WRONG D! -- > not the same;/

LoadError: syntax: extra token "IS" after end of expression

In [5]:
MYD(ts[1],10)

11×11 Array{Float64,2}:
    -27.5           -19.0536     …   1.53533e-6     1.28093e-6 
      9.98069         0.0            1.15365e-6     9.6196e-7  
     -3.45714        -6.89338       -1.42411e-6    -1.18581e-6 
      1.04415         1.70303        1.00144e-6     8.31702e-7 
     -1.56271        -2.38567       -2.99165e-6    -2.47339e-6 
    -44.8299        -66.4165     …  -0.000163099   -0.000133757
  -2754.3         -4016.52          -0.0194894     -0.01573    
 -23711.8        -34270.4           -0.365626      -0.284175   
 -85997.7            -1.23666e5     -4.01674       -2.7414     
     -1.67628e5      -2.40455e5      0.0          -29.3671     
     -1.95171e5      -2.79809e5  …  40.9831        27.5        

In [6]:
using JuMP
mdl = Model()
@variable(mdl, tf_var)
ts_JuMP, ωₛ_JuMP = create_intervals_JuMP(mdl,tf_var,Nck,Ni,τ,ω)
DJ=poldif_JuMP(mdl,ts_JuMP,Ni,Nck);
setvalue(tf_var, tf)

In [7]:
getvalue(ts_JuMP[2])

6-element Array{Any,1}:
 2.0    
 2.27952
 2.83282
 3.44631
 3.88579
 4.0    

In [8]:
tm

3-element Array{Float64,1}:
 0.0
 2.0
 4.0

In [9]:
tm = @NLexpression(mdl, [idx=1:Ni+1], (idx-1)*tf_var/Ni);
getvalue(tm)

3-element Array{Float64,1}:
 0.0
 2.0
 4.0

In [10]:
ts[2]

6-element Array{Float64,1}:
 2.0    
 2.27952
 2.83282
 3.44631
 3.88579
 4.0    

In [14]:
getvalue(DJ[1])

11×11 Array{Float64,2}:
 -25.25       34.4508     -14.4102    …  -5.678      8.10738  -5.0    
  -5.51997    -0.259405     8.32426       2.36547   -3.3692    2.07672
   1.2443     -4.48604     -0.283472     -1.91246    2.70663  -1.66597
  -0.502804    1.4828      -3.40753       1.8208    -2.54646   1.56332
   0.263226   -0.726584     1.29673      -1.94326    2.66097  -1.62626
  -0.159518    0.427308    -0.695489  …   2.32759   -3.06459   1.85781
   0.105735   -0.278793     0.434072     -3.26097    3.95081  -2.35712
  -0.0736453   0.192452    -0.292549      6.2693    -5.99646   3.44517
   0.0513909  -0.13362      0.20046      -3.36078   12.6751   -6.39464
  -0.0317445   0.0823339   -0.122733     -5.48338  -17.3462   21.7084 
   0.05       -0.129611     0.192936  …   7.06521  -55.442    50.0    

In [15]:
poldif(ts[1],1)

11×11 Array{Float64,2}:
 -25.25       34.4508     -14.4102    …  -5.678      8.10738  -5.0    
  -5.51997    -0.259405     8.32426       2.36547   -3.3692    2.07672
   1.2443     -4.48604     -0.283472     -1.91246    2.70663  -1.66597
  -0.502804    1.4828      -3.40753       1.8208    -2.54646   1.56332
   0.263226   -0.726584     1.29673      -1.94326    2.66097  -1.62626
  -0.159518    0.427308    -0.695489  …   2.32759   -3.06459   1.85781
   0.105735   -0.278793     0.434072     -3.26097    3.95081  -2.35712
  -0.0736453   0.192452    -0.292549      6.2693    -5.99646   3.44517
   0.0513909  -0.13362      0.20046      -3.36078   12.6751   -6.39464
  -0.0317445   0.0823339   -0.122733     -5.48338  -17.3462   21.7084 
   0.05       -0.129611     0.192936  …   7.06521  -55.442    50.0    

In [13]:
GOOD!!

LoadError: syntax: extra token "first" after end of expression

In [20]:
getvalue(ωₛ_JuMP[2])

5-element Array{Any,1}:
 0.08    
 0.446208
 0.623653
 0.562712
 0.287427

In [19]:
ωₛ[2]

5-element Array{Float64,1}:
 0.08    
 0.446208
 0.623653
 0.562712
 0.287427

In [ ]:
GOOD!!!